In [2]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import pathlib as pth
import scipy.stats as stat
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.magoptoffline as mg
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [3]:
tilt_adc_file = '/run/media/jleland/Seagate/data/magnum/2019-05-29_Leland/2019-05-29 17h 15m 42s 20TT_06696467906551624208.adc'
no_tilt_adc_file = '/run/media/jleland/Seagate/data/magnum/2019-05-29_Leland/2019-05-29 17h 18m 50s 20TT_06696468713881848757.adc'

In [4]:
os.chdir('/home/jleland/data/external/magnum/')
all_dataset = xr.open_dataset('all_meta_data.nc').max('ts_radial_pos')
shot_numbers = all_dataset.where(np.isfinite(all_dataset['adc_index']), drop=True)['shot_number'].values
shot_dataset = all_dataset.sel(shot_number=[111,112])

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [5]:
shot_dataset

<xarray.Dataset>
Dimensions:                 (shot_number: 2)
Coordinates:
  * shot_number             (shot_number) int32 111 112
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 116.0 117.0
    adc_time                (shot_number) datetime64[ns] ...
Data variables:
    adc_filename            (shot_number) object '2019-05-29 17h 15m 42s TT_06696467906551624208.adc' '2019-05-29 17h 18m 50s TT_06696468713881848757.adc'
    ts_density              (shot_number) float64 1.339e+19 1.148e+19
    ts_temperature          (shot_number) float64 3.086 2.305
    ts_d_density            (shot_number) float64 4.252e+16 8.294e+16
    ts_d_temperature        (shot_number) float64 0.03343 0.02121
    adc_folder              (shot_number) object '2019-05-29_Leland/' '2019-05-29_Leland/'
    adc_calibration_index   (shot_number) object '0' '0'
    adc_4_probe             (shot_number) object 'S' 'S'
    adc_5_probe             (shot_number) object 'L' 'L'
    ts_temp_max             (shot_number) float64 3.086 2.305
    ts_dens_max             (shot_number) float64 1.339e+19 1.148e+19
    shot_end_time           (shot_number) datetime64[ns] 2019-05-29T16:15:48 2019-05-29T16:18:56
    ts_fl                   (shot_number) bool True True
    shot_time               (shot_number) datetime64[ns] 2019-05-29T16:15:28 2019-05-29T16:18:36
    shot_source_current     (shot_number) float64 100.6 99.93
    shot_b_field            (shot_number) float64 0.7999 0.7999
    shot_tilt               (shot_number) float64 9.99 0.01195
    shot_target_pos         (shot_number) float64 -0.15 -0.15
    shot_target_rot         (shot_number) float64 -89.94 -89.94
    shot_target_pumpspeed   (shot_number) int32 82 82
    shot_source_pumpspeed   (shot_number) int32 82 82
    shot_heating_pumpspeed  (shot_number) int32 82 82
    shot_source_pos         (shot_number) float64 1.35 1.35
    shot_deuterium_gf       (shot_number) float64 0.00049 0.001431
    shot_hydrogen_gf        (shot_number) float64 7.598 7.601
    shot_helium_gf          (shot_number) float64 0.0 0.0
    shot_hydrogen_puffed    (shot_number) float64 -0.02963 -0.01565
    shot_helium_puffed      (shot_number) float64 -0.05167 -0.0535
    adc_timestamp           (shot_number) float64 6.696e+18 6.696e+18
    adc_4_coax              (shot_number) float64 1.0 1.0
    adc_4_shunt_resistance  (shot_number) float64 10.0 10.0
    adc_5_coax              (shot_number) float64 2.0 2.0
    adc_5_shunt_resistance  (shot_number) float64 10.0 10.0
    adc_calibration_fl      (shot_number) float64 0.0 0.0
    shot_fl                 (shot_number) float64 1.0 1.0
    acquisition_length      (shot_number) float64 10.0 10.0
    adc_end_time            (shot_number) datetime64[ns] 2019-05-29T16:15:42 2019-05-29T16:18:50
    adc_end_timestamp       (shot_number) float64 6.696e+18 6.696e+18
    adc_freqs               (shot_number) float64 1e+06 1e+06

In [6]:
CABLE_RESISTANCES = [1.7, 1.7, 1.7, 1.6]
PROBE_RESISTANCES = {
    'S': 1.0,
    'L': 1.0,
    'B': 1.0,
    'R': 1.8
}
FEEDTHROUGH_RESISTANCE = 1.25
INTERNAL_RESISTANCE = 6.09

In [13]:
magopters = []
mg.Magoptoffline._FOLDER_STRUCTURE = '/home/jleland/data/external/magnum/'

for shot in shot_dataset.shot_number.values:
    shot_dataarray = shot_dataset.sel(shot_number=shot)
    
    folder = str(shot_dataarray['adc_folder'].values)
    adc_file = str(shot_dataarray['adc_filename'].values)
    
    shunt_resistance = shot_dataarray['adc_4_shunt_resistance'].values
    
    probe_designations = (str(shot_dataarray['adc_4_probe'].values), str(shot_dataarray['adc_5_probe'].values))
    print(probe_designations)
    downsampling_factor = 1
    cabling_resistance = (CABLE_RESISTANCES[int(shot_dataarray['adc_4_coax'].values) - 1] + FEEDTHROUGH_RESISTANCE
                          + INTERNAL_RESISTANCE + PROBE_RESISTANCES[str(shot_dataarray['adc_4_probe'].values)])
    cabling_resistances = [cabling_resistance, cabling_resistance]
    
    magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistance, cabling_resistance=cabling_resistances)
    magopter._VOLTAGE_CHANNEL = 3
    magopter._PROBE_CHANNEL_3 = 4
    magopter._PROBE_CHANNEL_4 = 5
    magopter.prepare(down_sampling_rate=1, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None, homogenise_fl=False)
    
    magopters.append(magopter)


('S', 'L')
Running an offline magopter object, operating in offline mode.
('S', 'L')
Running an offline magopter object, operating in offline mode.


In [14]:
print(len(magopters))

2


In [18]:
probe_colours = {
    'L': '#4477AA',
    'S': '#EE6677',
    'B': '#228833',
    'R': '#CCBB44'
}

In [38]:
for magopter in magopters[0:1]:
    fig, axes = plt.subplots(2, figsize=[8,6], sharex=True)
    for i in range(2):
#         axes[i].plot(magopter.raw_time, magopter.current[i])
#         axes[i].set_title(f'{probe_designations[i]}')
        
#         fig, axes = plt.subplots(2)

        spectrum = np.fft.fft(magopter.current[i])
        freq = np.fft.fftfreq(len(magopter.current[i]), magopter.raw_time[1] - magopter.raw_time[0])
        pstve_f = np.where(freq > 0)
        
#         print(pstve_f.shape)
        
        axes[i].plot(freq[pstve_f], np.abs(spectrum)[pstve_f], 'x', label=f'{probe_designations[i]} Probe Spectrum', color=probe_colours[probe_designations[i]])
        axes[i].set_ylabel('Amplitude')
        
        axes[i].grid(which='both', axis='both')
        axes[i].set_xlim(0.01, 130000)
#         axes[0].set_ylim(1e-6, 1)
#         axes[i].set_xscale('log')
#         axes[i].set_yscale('log')
        axes[i].axvline(4.5e4, color='black', alpha=0.7, label='Filter', linestyle='dashed', lw=0.8)
        axes[i].axvspan(4.5e4, 1.3e5, color='grey', alpha=0.1)
        axes[i].legend()
    
    axes[1].set_xlabel('Frequency [Hz]')
    fig.tight_layout()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


Seems to be a peak (+ harmonics) at 51kHz - which is where we expected to see something from the power supply oscillations. Applying a filter at 45kHz might be a good idea. 